# Enrichissement d’un CSV avec l’API DPE

**Auteur :** Loick – Spécialiste GitHub & Jupyter  
**Public visé :** Data scientists / développeurs (niveau intermédiaire)

---

## 1. Introduction au sujet

Ce notebook explique pas à pas comment enrichir un fichier CSV contenant des coordonnées GPS avec les données du jeu de données **DPE Logements existants** de l’ADEME via son API publique.  
Nous verrons :
- Pourquoi et quand enrichir vos données par API  
- Les bonnes pratiques pour respecter les quotas  
- Le code Python complet et commenté  
- Comment analyser et valider les résultats obtenus  

---

## 2. Contexte théorique

Avant de plonger dans le code, quelques points clefs :

1. **Jeu de données DPE (`dpe03existant`)**  
   - Hébergé sur data.ademe.fr (Data Fair).  
   - Expose un endpoint `/lines` permettant de filtrer par géolocalisation (`geopoint`, `geo_distance`).  

2. **Paramètres géographiques**  
   - `geopoint` : latitude,longitude en WGS84 (EPSG:4326).  
   - `geo_distance` : rayon de recherche autour de ce point (en mètres ou autre unité Elasticsearch).  

3. **Quotas & limitations**  
   - Anonyme : 600 requêtes / 60 s → ~10 req/s.  
   - Authentifié : 1200 req / 60 s → ~20 req/s.  
   - Ajoutez un `sleep` (≥0.1 s) pour rester sous les limites.

4. **Workflow**  
   - Lecture du CSV par morceaux (chunksize)  
   - Cache local des appels pour éviter les doublons  
   - Sauvegarde incrémentale du CSV enrichi  

---

## 3. Implémentation pas à pas

### 3.1. Imports et configuration

In [ ]:
# %% Imports & config
import pandas as pd
import requests
import time
import os
from tqdm import tqdm

# Chemins
INPUT_FILE  = "merged_sales_data_dpe_enriched.csv"
OUTPUT_FILE = "merged_sales_data_dpe_enriched_complet.csv"

# Traitement par chunks
CHUNK_SIZE = 10_000

# API parameters
DISTANCE = 100         # rayon en mètre
SLEEP_BETWEEN_CALLS = 0.11
API_KEY = None         # ou votre clé ADEME

# Mapping des champs API → colonnes CSV
MAPPING_API_TO_DF = {
    "type_energie_principale_chauffage": "chauffage_energie",
    "type_installation_chauffage_n1":    "chauffage_systeme",
    "type_emetteur_installation_chauffage_n1": "chauffage_mode",
    "annee_construction":                "annee_construction",
    "etiquette_ges":                     "ges_class"
}
COLS_TO_FILL = list(MAPPING_API_TO_DF.values())

### 3.2. Détection du séparateur

In [ ]:
# %% Détection du séparateur CSV
def detect_delimiter(fp):
    with open(fp, 'r', encoding='utf-8') as f:
        line = f.readline()
    if ',' in line: return ','
    if ';' in line: return ';'
    if '\t' in line: return '\t'
    return ','

sep = detect_delimiter(INPUT_FILE)
print(f"Using separator: '{sep}'")

### 3.3. Fonction d’appel API avec cache

In [ ]:
# %% Fonction d'appel API
session = requests.Session()
cache = {}

def get_dpe_fields(lat, lon):
    """Retourne dict(fields) ou None."""
    try:
        lat, lon = float(lat), float(lon)
    except:
        return None
    key = (lat, lon)
    if key in cache:
        return cache[key]
    geopoint = f"{lat},{lon}"
    url = "https://data.ademe.fr/data-fair/api/v1/datasets/dpe03existant/lines"
    params = {"geopoint": geopoint, "geo_distance": DISTANCE, "size": 1}
    headers = {}
    if API_KEY: headers["X-API-Key"] = API_KEY

    try:
        r = session.get(url, params=params, headers=headers, timeout=5)
        r.raise_for_status()
        data = r.json()
        fields = data.get("results", [{}])[0].get("fields", None)
    except:
        fields = None

    cache[key] = fields
    return fields

### 3.4. Lecture, enrichissement et sauvegarde

In [ ]:
# %% Enrichissement chunk par chunk
if os.path.exists(OUTPUT_FILE):
    os.remove(OUTPUT_FILE)

reader = pd.read_csv(INPUT_FILE, sep=sep, chunksize=CHUNK_SIZE)
first = True

for chunk_idx, df in enumerate(reader, start=1):
    print(f"\n--- Chunk {chunk_idx} ---")
    # conversion coords
    df["mapCoordonneesLatitude"]  = pd.to_numeric(df["mapCoordonneesLatitude"],  errors="coerce")
    df["mapCoordonneesLongitude"] = pd.to_numeric(df["mapCoordonneesLongitude"], errors="coerce")

    # mask : au moins une colonne cible manquante
    mask = ~df[COLS_TO_FILL].notna().all(axis=1) & \
           df["mapCoordonneesLatitude"].notna() & df["mapCoordonneesLongitude"].notna()
    to_fill = df[mask]

    for idx in tqdm(to_fill.index, desc="Enrich"):
        lat = df.at[idx, "mapCoordonneesLatitude"]
        lon = df.at[idx, "mapCoordonneesLongitude"]
        fields = get_dpe_fields(lat, lon)
        if fields:
            for a, c in MAPPING_API_TO_DF.items():
                if pd.isna(df.at[idx, c]) and a in fields:
                    df.at[idx, c] = fields[a]
        time.sleep(SLEEP_BETWEEN_CALLS)

    # sauvegarde incrémentale
    if first:
        df.to_csv(OUTPUT_FILE, index=False, sep=sep)
        first = False
    else:
        df.to_csv(OUTPUT_FILE, mode="a", header=False, index=False, sep=sep)

## 4. Analyse des résultats

In [ ]:
# %% Vérifications
df_orig    = pd.read_csv(INPUT_FILE, sep=sep)
df_enriched= pd.read_csv(OUTPUT_FILE, sep=sep)

print(f"Orig : {len(df_orig)} lignes")
print(f"Enrichi : {len(df_enriched)} lignes")

for col in COLS_TO_FILL:
    print(f"{col} → {df_enriched[col].notna().sum():,} non-nuls")

## 5. Conclusion
	•	Workflow robuste : Le traitement par chunks, associé à un système de cache et à des pauses contrôlées, permet d’enrichir en toute fiabilité un gros fichier CSV via l’API DPE de l’ADEME, tout en respectant les quotas.
	•	Scalabilité :
	•	Lecture chunkée → évite de charger tout le fichier en mémoire.
	•	Cache des coordonnées → minimise les appels redondants.
	•	Modularité :
	•	Paramètres (taille de chunk, distance, pause API) facilement ajustables.
	•	Mapping API ↔ colonnes CSV simple à étendre à d’autres champs.
	•	Validation facile :
	•	Scripts de vérification post‑traitement (comparaison du nombre de lignes, comptage des valeurs non-nulles) pour garantir que tous les enregistrements ont bien été parcourus et enrichis.
	•	Ressources complémentaires :
	•	Documentation API DPE : https://data.ademe.fr/data-fair/api/v1/datasets/dpe03existant
	•	Pandas : https://pandas.pydata.org/
	•	Matplotlib : https://matplotlib.org/
	•	Seaborn : https://seaborn.pydata.org/